Exercise: Blue/Green Deployments
-----------------

In this exercise we will perform a blue/green deployment.

For this purpose, another deployment is created with the new version of the BPMN frontend and switched back and forth between the versions by changing the control of the service.

This happens in a separate namespace to be able to display the results in a targeted manner:

In [ ]:
! kubectl create namespace bluegreen

Deployment and Services has been split into separate files. The Ingress resource, omitted for simplicity.

A version specification has been added to the labels and the image:

    apiVersion: v1
    kind: Service
    metadata:
      name: bpmn-frontend
      labels:
        app: bpmn-frontend
    spec:
      ...
      selector:
        app: bpmn-frontend
        version: "0.2"    
        
- - -

    apiVersion: apps/v1
    kind: Deployment
    metadata:
      name: bpmn-frontend
      version: 0.2
    ...
    spec:
      containers:
      - name: bpmn-frontend
        image: registry.gitlab.com/mc-b/misegr/misegr/bpmn-frontend:V0.2
        imagePullPolicy: IfNotPresent       



In [ ]:
! kubectl apply -f 09-4-Deployment/bpmn-frontend-deployment-green.yaml --namespace bluegreen
! kubectl apply -f 09-4-Deployment/bpmn-frontend-service-green.yaml --namespace bluegreen

The result, for the time being, does not differ from a normal deployment

In [ ]:
! kubectl get all --namespace bluegreen
! echo "BPMN Frontend: http://$(cat ~/work/server-ip):$(kubectl get service --namespace bluegreen bpmn-frontend -o=jsonpath='{ .spec.ports[0].nodePort }')/frontend/index.html"

- - -

For the **blue** deployment we only create a deployment but no service:

In [ ]:
! kubectl apply -f 09-4-Deployment/bpmn-frontend-deployment-blue.yaml --namespace bluegreen
! kubectl get all --namespace bluegreen -o wide

The service is heading towards the **green** pods.

Only after changing the selector to the **blue** version will this be visible to the user

        apiVersion: v1
        kind: Service
        metadata:
          name: bpmn-frontend
          labels:
            app: bpmn-frontend
        spec:
          ...
          selector:
            app: bpmn-frontend
            version: "1.0"
        

In [ ]:
! kubectl apply -f 09-4-Deployment/bpmn-frontend-service-blue.yaml --namespace bluegreen

The selector does not change anything about the number of resources

In [ ]:
! kubectl get all --namespace bluegreen -o wide

- - -
After the successful tests of the new version we can delete the **green** deployment.

In [ ]:
! kubectl delete -f 09-4-Deployment/bpmn-frontend-deployment-green.yaml --namespace bluegreen
! kubectl get all --namespace bluegreen

- - -

Clean up

In [ ]:
! kubectl delete namespace bluegreen